In [77]:
import pandas as pd
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
import nltk
import re
analyser = SentimentIntensityAnalyzer()

In [78]:
data=pd.read_csv("skirt-flipkart-final-final.csv")
data.head()
data=data[['URL','ID','ITEM','STARS','NUMBER OF RATINGS','NUMBER OF REVIEWS', 'LIST OF REVIEWS']]

In [79]:
data.rename(columns = {'ID':'id','ITEM':'desc', 'LIST OF REVIEWS':'reviews','NUMBER OF REVIEWS':'num_reviews','STARS':'stars','NUMBER OF RATINGS':'num_ratings'}, inplace=True)
data.head()

,URL,id,desc,stars,num_ratings,num_reviews,reviews
0,http://flipkart.com/vikalps-printed-women-regu...,0,Women Printed Regular Maroon Skirt,3.8,536,83,"['nice skirt.. n febric is also good, go for i..."
1,http://flipkart.com/frionkandy-printed-women-w...,1,Women Printed Wrap Around Black Skirt,4.1,1825,297,"['nice quality in this price', 'lovely product..."
2,http://flipkart.com/vikalps-embroidered-women-...,2,Women Embroidered A-line White Skirt,4.1,60,14,"['Super', 'Wow', 'Quality parameters of skirt ..."
3,http://flipkart.com/unique-choice-printed-wome...,3,Women Printed A-line Black Skirt,4.1,273,44,"['really worth for money', 'Amazing product.. ..."
4,http://flipkart.com/unique-choice-printed-wome...,4,Women Printed Wrap Around White Skirt,4.1,223,34,"['Good items thanks', 'Superb', 'Fantastic']"


In [80]:
data.reviews = data.reviews.str.lower()
data.reviews = data.reviews.str.replace('\n','').str.replace('[\'!"#$%&\()*+,-./:;<=>?@[\\]^_`{|}~]','')

In [81]:
data['vader_score'] = 1.0
for ind in data.index: 
    rev=data['reviews'][ind]
    x = rev.split()
    sum_score=0.0
    for i in x:
        score = analyser.polarity_scores(i)
        sum_score=sum_score+score['compound']
    data['vader_score'][ind] =sum_score

C:\Users\Miloni Mittal\Anaconda3\ANACONDA\lib\site-packages\ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':


In [82]:
total_star_givers = data['num_ratings'].sum()     
total_review_givers = data['num_reviews'].sum()

In [83]:
data['final_score'] = ((data['stars']*data['num_ratings'])/total_star_givers)+((data['vader_score']*data['num_reviews'])/total_review_givers)

In [84]:
data.head()

,URL,id,desc,stars,num_ratings,num_reviews,reviews,vader_score,final_score
0,http://flipkart.com/vikalps-printed-women-regu...,0,Women Printed Regular Maroon Skirt,3.8,536,83,nice skirt n febric is also good go for it val...,2.5933,0.036874
1,http://flipkart.com/frionkandy-printed-women-w...,1,Women Printed Wrap Around Black Skirt,4.1,1825,297,nice quality in this price lovely product must...,2.4172,0.130452
2,http://flipkart.com/vikalps-embroidered-women-...,2,Women Embroidered A-line White Skirt,4.1,60,14,super wow quality parameters of skirt is good ...,1.6257,0.004231
3,http://flipkart.com/unique-choice-printed-wome...,3,Women Printed A-line Black Skirt,4.1,273,44,really worth for money amazing product totally...,4.1017,0.024543
4,http://flipkart.com/unique-choice-printed-wome...,4,Women Printed Wrap Around White Skirt,4.1,223,34,good items thanks superb fantastic,2.0631,0.014731


In [90]:
data.to_csv('skirts_csv_final.csv')

In [85]:
import nltk
import re
dict_bigram={}
item_bigram={}
# word_data = "The best performance can bring in sky high success."
for ind in data.index:
    d=data['desc'][ind][6:-5]
    d= re.sub(r'[^\w\s]', '', d) 
    nltk_tokens = nltk.word_tokenize(d)
    arr=list(nltk.bigrams(nltk_tokens))
    for i in arr:
        if i in dict_bigram:
            dict_bigram[i]=dict_bigram[i]+1
            item_bigram[" ".join(i)].append(ind)
        else:
            dict_bigram[i]=1
            item_bigram[" ".join(i)]=[]
            item_bigram[" ".join(i)].append(ind)
print(item_bigram) 


{'Printed Regular': [0, 10, 25, 26, 58, 68, 89, 96, 132, 137, 142, 144, 161, 173, 177, 194, 200, 203, 217, 221, 222, 250, 253, 282, 290, 303], 'Regular Maroon': [0, 86, 148, 196, 203, 238, 294], 'Printed Wrap': [1, 4, 6, 17, 19, 20, 29, 30, 32, 36, 56, 66, 72, 99, 104, 108, 109, 111, 112, 131, 138, 140, 145, 147, 154, 158, 160, 166, 174, 178, 204, 205, 206, 208, 218, 227, 229, 235, 240, 260, 267, 268, 280, 287, 295, 298, 304, 308, 311], 'Wrap Around': [1, 4, 6, 13, 16, 17, 19, 20, 21, 29, 30, 32, 36, 50, 51, 56, 60, 66, 69, 72, 76, 99, 100, 101, 104, 108, 109, 111, 112, 119, 120, 131, 138, 140, 145, 147, 154, 158, 160, 166, 174, 178, 191, 204, 205, 206, 208, 218, 227, 229, 235, 240, 260, 267, 268, 280, 287, 295, 298, 304, 308, 311], 'Around Black': [1, 17, 101, 260, 308], 'Embroidered Aline': [2, 22], 'Aline White': [2, 141, 168, 175, 230, 300], 'Printed Aline': [3, 9, 33, 42, 43, 49, 53, 79, 141, 168, 171, 226, 239, 242, 244, 300], 'Aline Black': [3, 18, 22, 33, 42, 49, 53, 79, 188, 2

In [86]:
dict_bigram_sorted= sorted(dict_bigram, key=dict_bigram.get, reverse=True)
print(dict_bigram_sorted)

[('Wrap', 'Around'), ('Printed', 'Wrap'), ('Printed', 'Flared'), ('Floral', 'Print'), ('Printed', 'Regular'), ('Solid', 'Regular'), ('Solid', 'Flared'), ('Dark', 'Blue'), ('Around', 'Blue'), ('Regular', 'Black'), ('Printed', 'Aline'), ('Flared', 'Multicolor'), ('Aline', 'Black'), ('Self', 'Design'), ('Solid', 'Aline'), ('Around', 'Multicolor'), ('Print', 'Regular'), ('Regular', 'Red'), ('Flared', 'Black'), ('Regular', 'Dark'), ('Print', 'Wrap'), ('Flared', 'Blue'), ('Print', 'Flared'), ('Solid', 'Pencil'), ('Around', 'Red'), ('White', 'Black'), ('Flared', 'Pink'), ('Print', 'Aline'), ('Aline', 'Blue'), ('Regular', 'Maroon'), ('Regular', 'Blue'), ('Regular', 'Pink'), ('Animal', 'Print'), ('Aline', 'White'), ('Around', 'White'), ('Flared', 'White'), ('Flared', 'Maroon'), ('Pencil', 'Black'), ('Flared', 'Dark'), ('Around', 'Black'), ('Design', 'Flared'), ('Printed', 'Gathered'), ('Design', 'Gathered'), ('Regular', 'Green'), ('Aline', 'Dark'), ('Gathered', 'Pink'), ('Striped', 'Regular'), 

In [87]:
row=dict_bigram_sorted[0:5]+dict_bigram_sorted[-5:]
print(row)

[('Wrap', 'Around'), ('Printed', 'Wrap'), ('Printed', 'Flared'), ('Floral', 'Print'), ('Printed', 'Regular'), ('Gathered', 'Brown'), ('Checkered', 'Gathered'), ('Gathered', 'Black'), ('Peplum', 'Light'), ('Gathered', 'Maroon')]


In [88]:
res = [] 
for el in row: 
    sub=" ".join(el)
    res.append(list(sub.split("*"))) 
print(res)

[['Wrap Around'], ['Printed Wrap'], ['Printed Flared'], ['Floral Print'], ['Printed Regular'], ['Gathered Brown'], ['Checkered Gathered'], ['Gathered Black'], ['Peplum Light'], ['Gathered Maroon']]


In [89]:
import csv
with open('skirt_top_bottom.csv', 'w', newline='') as file:
    res = [] 
    for el in row: 
        sub=" ".join(el)
        res.append(list(sub.split("*"))) 
    for i in range(len(res)):
        res[i]=res[i]+item_bigram[res[i][0]]
    print(res)
    writer = csv.writer(file)
    writer.writerows(res)

[['Wrap Around', 1, 4, 6, 13, 16, 17, 19, 20, 21, 29, 30, 32, 36, 50, 51, 56, 60, 66, 69, 72, 76, 99, 100, 101, 104, 108, 109, 111, 112, 119, 120, 131, 138, 140, 145, 147, 154, 158, 160, 166, 174, 178, 191, 204, 205, 206, 208, 218, 227, 229, 235, 240, 260, 267, 268, 280, 287, 295, 298, 304, 308, 311], ['Printed Wrap', 1, 4, 6, 17, 19, 20, 29, 30, 32, 36, 56, 66, 72, 99, 104, 108, 109, 111, 112, 131, 138, 140, 145, 147, 154, 158, 160, 166, 174, 178, 204, 205, 206, 208, 218, 227, 229, 235, 240, 260, 267, 268, 280, 287, 295, 298, 304, 308, 311], ['Printed Flared', 5, 12, 75, 87, 88, 103, 107, 114, 115, 139, 153, 155, 157, 159, 167, 170, 190, 192, 198, 231, 234, 236, 237, 254, 255, 265, 266, 277, 283, 305, 312], ['Floral Print', 7, 13, 24, 27, 44, 46, 48, 50, 51, 55, 61, 77, 82, 91, 98, 105, 116, 129, 130, 134, 150, 172, 185, 215, 224, 245, 263, 271, 281, 284, 296], ['Printed Regular', 0, 10, 25, 26, 58, 68, 89, 96, 132, 137, 142, 144, 161, 173, 177, 194, 200, 203, 217, 221, 222, 250, 253,